# San Francisco Airbnb Data Analysis
## Data Cleaning
A Proposed Capstone Project for TDI

Haoming Jin

In [129]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
import matplotlib.pyplot as plt
import seaborn

In [130]:
df = pd.DataFrame()
filelist = ['2020-04-07','2020-03-13','2020-02-12','2020-01-04','2020-01-02','2019-12-04','2019-11-01','2019-10-14',
            '2019-09-12','2019-08-06','2019-07-08','2019-06-02','2019-05-03','2019-04-03']
for file in filelist:
    try:
        url = "http://data.insideairbnb.com/united-states/ca/san-francisco/"+file+"/data/listings.csv.gz"
        resp = urlopen(url)
        df_temp = pd.read_csv(resp, compression = 'gzip')
        df = df.append(df_temp)
        # For the records from the same id, we take the newer record
        df = df[~df.id.duplicated()]
    except:
        print(file, ' fail')

C:\Users\Kami\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (61,62) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Cleaning

I am first dropping columns that we definitely not using, they are empty meaningless or can be easily get from other data in the dataset.

In [132]:
drop_list = ['listing_url','scrape_id','experiences_offered','thumbnail_url','medium_url','xl_picture_url',
             'host_thumbnail_url', 'neighbourhood_group_cleansed', 'city','state','market','smart_location',
            'country_code', 'country', 'square_feet', 'minimum_minimum_nights', 'maximum_minimum_nights',
             'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm','maximum_nights_avg_ntm',
             'calendar_last_scraped','jurisdiction_names']
df_dropped = df.drop(labels = drop_list, axis = 1)

Due to the large amount of columns we will first clean items that are relavant or easy to clean.
1. change some rows of t/f to boolean type
2. change prices to float type.

In [133]:
cols_to_bool = ['host_is_superhost','host_has_profile_pic','host_identity_verified','has_availability','instant_bookable',
                'is_business_travel_ready','require_guest_profile_picture','require_guest_phone_verification']
cols_to_price = ['host_response_rate','host_acceptance_rate',
                 'price','weekly_price','monthly_price','security_deposit','cleaning_fee','extra_people']
cols_to_datetime = ['last_scraped', 'first_review','last_review']
cols_to_perc = ['host_response_rate','host_acceptance_rate']
cols_to_str = ['id','host_id']

In [134]:
df_dropped[cols_to_bool] = (df_dropped[cols_to_bool] == 't')

In [135]:
def clean_currency(x):
    if isinstance(x, str):
        return x.replace('$','').replace(',','')
    else: return x
for col in cols_to_price:
    df_dropped[col] = df_dropped[col].apply(clean_currency).astype('float')

In [136]:
df_dropped[cols_to_datetime] = df_dropped[cols_to_datetime].apply(lambda x: pd.to_datetime(x))

In [137]:
def clean_perc(x):
    if isinstance(x, str):
        return x.replace('%','')
    else: return x
for col in cols_to_perc:
    df_dropped[col] = df_dropped[col].apply(clean_perc).astype('float')/100

In [138]:
for col in cols_to_str:
    df_dropped[col] = df_dropped[col].astype(str)

In [139]:
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12074 entries, 0 to 7275
Data columns (total 83 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   id                                            12074 non-null  object        
 1   last_scraped                                  12074 non-null  datetime64[ns]
 2   name                                          12074 non-null  object        
 3   summary                                       11730 non-null  object        
 4   space                                         10064 non-null  object        
 5   description                                   11947 non-null  object        
 6   neighborhood_overview                         8759 non-null   object        
 7   notes                                         6860 non-null   object        
 8   transit                                       8106 non-null   objec

In [140]:
df_dropped.describe()

,host_response_rate,host_acceptance_rate,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
count,10364.000000,8516.000000,12070.000000,12070.000000,12074.000000,12074.000000,12074.000000,12064.000000,12065.000000,12007.000000,...,8762.000000,8759.000000,8763.000000,8759.000000,8759.000000,12074.000000,12074.000000,12074.000000,12074.000000,8891.000000
mean,0.949767,0.880082,115.239685,115.239685,37.766015,-122.429654,3.153470,1.409773,1.351513,1.746315,...,9.569276,9.818587,9.783864,9.678845,9.352552,25.947656,20.978218,4.004721,0.779443,1.641264
std,0.146818,0.196440,428.131384,428.131384,0.023899,0.027050,1.983336,0.923997,0.963192,1.263385,...,0.894532,0.671810,0.738809,0.742596,0.928804,64.851421,62.305973,10.888533,4.035210,1.936059
min,0.000000,0.000000,0.000000,0.000000,37.704170,-122.513060,1.000000,0.000000,0.000000,0.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.010000
25%,1.000000,0.820000,1.000000,1.000000,37.751243,-122.442717,2.000000,1.000000,1.000000,1.000000,...,9.000000,10.000000,10.000000,10.000000,9.000000,1.000000,0.000000,0.000000,0.000000,0.260000
50%,1.000000,0.980000,2.000000,2.000000,37.770050,-122.423720,2.000000,1.000000,1.000000,1.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,2.000000,1.000000,0.000000,0.000000,0.900000
75%,1.000000,1.000000,12.000000,12.000000,37.785850,-122.410360,4.000000,1.500000,2.000000,2.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,9.000000,2.000000,2.000000,0.000000,2.410000
max,1.000000,1.000000,2347.000000,2347.000000,37.828790,-122.367020,16.000000,14.000000,30.000000,30.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,301.000000,301.000000,87.000000,36.000000,31.020000


In [142]:
df_dropped.to_pickle('airbnb_SF_2019_04_to_2020_04.pkl')